# Pure Storage Provisioning

The goal of this notebook is to document the Python scripts used to proof how to use FlashArray and APIs during the block POC at Roche

This notebook follow the process defined at the POC where hosts and volumes are created, mapped, snapshots are reated, copied to new volumes, etc. 

In this example we have two Flash Arrays and two ESX hosts to be mapped.

Also here you have some documentation about Python module:

* __[Documentation Overview](http://pure-storage-python-rest-client.readthedocs.io/en/latest/)__
* __[Installation Instructions](http://pure-storage-python-rest-client.readthedocs.io/en/latest/installation.html)__
* __[purestorage python API glossary](http://pure-storage-python-rest-client.readthedocs.io/en/latest/api.html)__
* __[rest-client](https://github.com/purestorage/rest-client/blob/master/purestorage/purestorage.py)__



## Flash Array Definitions & Conections

Here you will be using your IP addresses, your array names, and your tokens. 


In [16]:
ArrayName1="se-emea-ebc-fam20"
IP1='10.225.112.80'
token1='6f203f95-3539-xxxx-xxxx-2005c3c1468c'

ArrayName2="se-emea-ebc-fam20"
IP2='10.225.112.180'
token2='c070a7f6-b57e-xxxx-xxxx-f60a08fbebd8'


volume_prefix = "POD1::CCX-"
volume_size_default = "16T"
threshold = 80

In [17]:
import purestorage

import urllib3
# We're using a version that warns us about insecure requests, and we don't want to see that noise.
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


In [18]:
try:
    array1 = purestorage.FlashArray(IP1, api_token=token1)

except: 
    print ("Issue connecting with Array: %s " % IP1)
    raise

print ("connected to Array: %s " % IP1)

    
info1 = array1.get() 
if (info1["array_name"] != ArrayName1):
    print "There is some mis-configuration because array name does not match"
    raise
    
try:
    array2 = purestorage.FlashArray(IP2, api_token=token2)


except: 
    print ("Issue connecting with Array: %s " % IP2)
    raise
    
print ("connected to Array: %s " % IP2)

info2 = array2.get() 
if (info2["array_name"] != ArrayName2):
    print "There is some mis-configuration because array name does not match"
    raise


connected to Array: 10.225.112.80 
connected to Array: 10.225.112.180 


## Host Creation

In [19]:
HostAttributes = {}
HostAttributes["personality"] = 'esxi'

HostGroupName = 'CLUSTERCC'

#Define here the host names that belong to the host group
HostNames = {}
HostNames["hostlist"] = ['CChost5at','CChost5bt']

#Now insert the wwwn for each host
WWN = {"CChost5at" : {"wwnlist" : ["91:42:53:44:55:66:77:85", "92:43:54:55:66:77:88:95"]},
       "CChost5bt" : {"wwnlist" : ["9A:42:53:44:55:66:77:85", "9B:43:54:55:66:77:88:95"]}
      }

#Define where each host is located. Their prefered array will be the one in the same data center
PreferedArrays = { "CChost5a" : {"prefered array" : "IBERIALAB1"},
                   "CChost5b" : {"prefered array" : "IBERIALAB2"}
      }


Create new Hosts en each of the Flash Arrays

Using the hosts we have previously defined, we are going to use each Flash Array connection to create a new host and set their Properties

In [20]:
if HostNames is not None:   # we have a list of hosts to be connected
    for host in HostNames["hostlist"]:
        wwnlist_host = WWN[host]
                
        try:
            h_info = array1.create_host(host, **wwnlist_host)
            print ("Adding Host: %s to Array: %s" % (host, ArrayName1))
            h_info = array1.set_host(host, **HostAttributes) 
            print ("Setting Attributre: %s on Host: %s and Array: %s" % (HostAttributes["personality"], host, ArrayName1))            
                                     
        except:
            print ("Issue adding host %s at array %s" % (host, ArrayName1))
            raise
     
        try:
            h_info = array2.create_host(host, **wwnlist_host)
            print ("Adding Host: %s to Array: %s" % (host, ArrayName2))
            h_info = array2.set_host(host, **HostAttributes) 
            print ("Setting Attributre: %s on Host: %s and Array: %s" % (HostAttributes["personality"], host, ArrayName2))            

        except:
            print ("Issue adding host %s at array %s" % (host, ArrayName2))
            raise


Adding Host: CChost5at to Array: se-emea-ebc-fam20-a
Setting Attributre: esxi on Host: CChost5at and Array: se-emea-ebc-fam20-a
Adding Host: CChost5at to Array: se-emea-ebc-fam20-b
Setting Attributre: esxi on Host: CChost5at and Array: se-emea-ebc-fam20-b
Adding Host: CChost5bt to Array: se-emea-ebc-fam20-a
Setting Attributre: esxi on Host: CChost5bt and Array: se-emea-ebc-fam20-a
Adding Host: CChost5bt to Array: se-emea-ebc-fam20-b
Setting Attributre: esxi on Host: CChost5bt and Array: se-emea-ebc-fam20-b


## Create a Host Group for Clustered Systems

Now we add the hosts into the host group we have defined

In [21]:
try:
    hgroup = array1.create_hgroup(HostGroupName, **HostNames)
    print ("Host Group: %s created in Array: %s" % (HostGroupName, ArrayName1))
except:
    print ("Issue creating Host Group %s in Array %s" % (HostGroupName, ArrayName1))
    raise
 
try:
    hgroup = array2.create_hgroup(HostGroupName, **HostNames)
    print ("Host Group: %s created in Array: %s" % (HostGroupName, ArrayName2))
except:
    print ("Issue creating Host Group %s in Array %s" % (HostGroupName, ArrayName2))
    raise


Host Group: CLUSTERCC created in Array: se-emea-ebc-fam20-a
Host Group: CLUSTERCC created in Array: se-emea-ebc-fam20-b


## Storage Provisioning


First, we are going to define again the  variables that we may need for this provisioning

If you have not done it already, execue the cell with the  Array Definition and the Cell to connect to the Flash Arrays

In [22]:

Pod = 'POD1'

LUNS_to_provision=['16T','16T']

### Check Capacity

Let´s make sure that each array do not pass the threshold defined. You can modify that value yourself and even avoid this section

In [23]:
array_info1 = array1.get(space="true")
for info in array_info1:
    capacity = info["capacity"]/1024/1024/1024
    total = (info["total"]/1024/1024/1024)
    utilization = float(100*total/capacity)
    print ("Array: %s" % (ArrayName1))
    print ("Total Capacity: %d GiB, Total Used: %d GiB" % (capacity, total))
    print ("Percentage capacity used: %d percent" % (utilization))

    
if utilization > threshold:
        print ("Space utilization is %d, make sure you have enough space before creating volumes" % utilization)
        print ("Volume not created")
        exit()
          

array_info2 = array2.get(space="true")
for info in array_info2:
    capacity = info["capacity"]/1024/1024/1024
    total = (info["total"]/1024/1024/1024)
    utilization = float(100*total/capacity)
    print ("Array: %s" % (ArrayName2))
    print ("Total Capacity: %d GiB, Total Used: %d GiB" % (capacity, total))
    print ("Percentage capacity used: %d percent" % (utilization))

if utilization > threshold:
        print ("Space utilization is %d, make sure you have enough space before creating volumes" % utilization)
        print ("Volume not created")
        exit()
          

Array: se-emea-ebc-fam20-a
Total Capacity: 5693 GiB, Total Used: 3261 GiB
Percentage capacity used: 57 percent
Array: se-emea-ebc-fam20-b
Total Capacity: 5691 GiB, Total Used: 1832 GiB
Percentage capacity used: 32 percent


### Create Volumes

The voume name is going to be given based on the Host Group Name and in the last 4 numbers for the LUN ID. Therfore, we first create a volume using a temporry name, then get their ID, compose the new name and rename it.

We store the name of the LUNs created into a list so we can map it later to the host group

In [24]:
LUNS_created = []
LUNS_copied = []

TempName = Pod + "::" + "Temporal"

for LUN in LUNS_to_provision:
    try:
        newvol = array1.create_volume(TempName, LUN)
    except:
        print "Error while creating the volume"
        raise    
    
    serial = newvol["serial"]
    newname = Pod + "::" + HostGroupName + "-" + serial[-4:]
    
    rename = array1.rename_volume(TempName, newname)
    
    print ("Created new volume: {} of size {}").format(newname, LUN)

    LUNS_created.append(newname)        

Created new volume: POD1::CLUSTERCC-E4C7 of size 16T
Created new volume: POD1::CLUSTERCC-E4C8 of size 16T


### Connect the new volumes to the host group

Now we take the list of the volumes created and we map it to the Host Group

In [25]:

for volume in LUNS_created:

    try:
        array1.connect_hgroup(HostGroupName, volume)
        array2.connect_hgroup(HostGroupName, volume)
        print ("Volume: %s, conected to Host Group: %s from both arrays" % (volume, HostGroupName))
    except:
        print ("There was some issue connecting Host Group: %s, with volume %s. Please check" % (HostGroupName, volume))
        raise


Volume: POD1::CLUSTERCC-E4C7, conected to Host Group: CLUSTERCC from both arrays
Volume: POD1::CLUSTERCC-E4C8, conected to Host Group: CLUSTERCC from both arrays


## Resize a Volume

Let´s increase the size of the volumes created previously

In [26]:
NEW_SIZE='20T'

for volume in LUNS_created:

    try:
        array1.extend_volume(volume, NEW_SIZE)
    except:
        print ("There was some issue resizing volume %s . Please check" % (volume))
        raise

## Create a Snapshot

In [29]:
for volume in LUNS_created:

    try:
        array1.create_snapshot(volume)
    except:
        print ("There was some issue resizing volume %s . Please check" % (volume))
        raise
        
for volume in LUNS_created:
    
    list = array2.get_volume(volume, snap=True)
    
    for v in  list:
        print "Snapshot: " , v["name"], "from:", v["source"]

Snapshot:  POD1::CLUSTERCC-E4C7.1 from: POD1::CLUSTERCC-E4C7
Snapshot:  POD1::CLUSTERCC-E4C7.2 from: POD1::CLUSTERCC-E4C7
Snapshot:  POD1::CLUSTERCC-E4C7.3 from: POD1::CLUSTERCC-E4C7
Snapshot:  POD1::CLUSTERCC-E4C8.1 from: POD1::CLUSTERCC-E4C8
Snapshot:  POD1::CLUSTERCC-E4C8.2 from: POD1::CLUSTERCC-E4C8
Snapshot:  POD1::CLUSTERCC-E4C8.3 from: POD1::CLUSTERCC-E4C8


## Copy a Snapshot

Let´s get the first snapshot for the first LUN an clone it

In [30]:
volume = LUNS_created[0]

list = array2.get_volume(volume, snap=True)

snaptoclone = list[0]["name"]
newname =  list[0]["source"]+"-Copy"

array1.copy_volume(snaptoclone, newname)

LUNS_copied.append(newname)    

## Cleanup

In [31]:
# Disconnect volume
HN = {}
HN["hostlist"] = []

for volume in LUNS_copied:    
    try:
        array1.destroy_volume(volume)
    except:
        print ("We got some issues")
        
        
for volume in LUNS_created:    
    try:
        array1.disconnect_hgroup(HostGroupName, volume)
        array2.disconnect_hgroup(HostGroupName, volume)
        
        array1.destroy_volume(volume)
        
 
        
        print ("Volume: %s, disconected from Host Group: %s from both arrays" % (volume, HostGroupName))
    except:
        print ("There was some issue disconnecting Host Group: %s, with volume %s. Please check" % (HostGroupName, volume))
        raise
        
        
array1.set_hgroup(HostGroupName, **HN)
array2.set_hgroup(HostGroupName, **HN)
                
array1.delete_hgroup(HostGroupName)
array2.delete_hgroup(HostGroupName)
        
    
if HostNames is not None:   # we have a list of hosts to be connected
    for host in HostNames["hostlist"]:
        array1.delete_host(host)
        array2.delete_host(host)
        
        
        

Volume: POD1::CLUSTERCC-E4C7, disconected from Host Group: CLUSTERCC from both arrays
Volume: POD1::CLUSTERCC-E4C8, disconected from Host Group: CLUSTERCC from both arrays
